In [17]:
import sqlite3 as sql
import pandas as pd

In [18]:
class sqlite_con_queries_inserts:
    # Cursor is now created with class
    def __init__(self, path, queries):
        self.path = path
        self.connection = self.create_connection()
        self.cursor = self.connection.cursor()
        self.queries = queries

    # Connection function
    def create_connection(self):
        connection = None
        try:
            connection = sql.connect(self.path)
            print("Connection to SQLite DB successful")
        except sql.Error as e:
            print(f"The error '{e}' occurred")
        return connection

    # Query function inputting cursor with a list of queries.
    def execute_query(self, data=None, print_message=True):
        try:
            for query in self.queries:
                if data:
                    self.cursor.execute(query, data)
                else:
                    self.cursor.execute(query)
            # Commit only if all statements succeeded
            self.cursor.connection.commit()
            if print_message:
                print("Queries executed successfully")
        except sql.Error as e:
            print("An error occurred:", e)
            # Rollback the transaction in case of error
            self.cursor.connection.rollback()
    
    # Inerts static function
    @staticmethod
    def execute_inserts(table, query, path):
        # Convert DataFrame rows to list of dictionaries
        my_list = table.to_dict(orient='records')
        # Iterate through the list of dictionaries and insert data
        for row in my_list:
            try:
                sqlite_con_queries_inserts(path, query).execute_query(data=row, print_message=False)
            except sql.Error as e:
                print("An error occurred:", e)
        print("Insert query executed successfully")

In [ ]:
# Create connection and cursor
path = 'TRL1_Database.sqlite'
queries = ""
conn = sqlite_con_queries_inserts(path, queries).create_connection()

In [ ]:
# CREATE TABLE queries in related order
create_table_queries = [
"""
CREATE TABLE IF NOT EXISTS airliners (
  airline_number VARCHAR PRIMARY KEY,
  name VARCHAR,
  location_country VARCHAR,
  location_region VARCHAR,
  location_city VARCHAR
);
""",
"""
CREATE TABLE IF NOT EXISTS suppliers (
  company_number VARCHAR PRIMARY KEY,
  name VARCHAR NOT NULL,
  location VARCHAR
);
""",
"""
CREATE TABLE IF NOT EXISTS software (
  software_number VARCHAR PRIMARY KEY,
  company_number VARCHAR,
  co_company_number VARCHAR,
  name VARCHAR,
  apps_features VARCHAR,
  FOREIGN KEY (company_number) REFERENCES suppliers(company_number)
);
""",
"""
CREATE TABLE IF NOT EXISTS hardware (
  hardware_number VARCHAR PRIMARY KEY,
  company_number VARCHAR,
  co_company_number VARCHAR,
  name VARCHAR,
  features VARCHAR,
  FOREIGN KEY (company_number) REFERENCES suppliers(company_number)
);
""",
"""
CREATE TABLE IF NOT EXISTS airline_software_relation (
  relation_number VARCHAR PRIMARY KEY,
  airline_number VARCHAR,
  software_number VARCHAR,
  relation_description VARCHAR,
  date TIMESTAMP,
  FOREIGN KEY (airline_number) REFERENCES airliners(airline_number),
  FOREIGN KEY (software_number) REFERENCES software(software_number)
);
""",
"""
CREATE TABLE IF NOT EXISTS airline_hardware_relation (
  relation_number VARCHAR PRIMARY KEY,
  airline_number VARCHAR,
  hardware_number VARCHAR,
  relation_description VARCHAR,
  date TIMESTAMP,
  FOREIGN KEY (airline_number) REFERENCES airliners(airline_number),
  FOREIGN KEY (hardware_number) REFERENCES hardware(hardware_number)
);
"""
]
path = 'TRL1_Database.sqlite'
queries = create_table_queries
# Call the function to create the tables
sqlite_con_queries_inserts(path, queries).execute_query()

In [ ]:
airliner_db = pd.read_excel('___.xlsx', sheet_name='___', index_col=None)
airliner_db = airliner_db.rename(columns={"___": "___"})
airliner_db

In [ ]:
table = airliner_db
# The column names in the airliner_db are renamed to match the tables in SQLite.
insert_stations = [
"""
INSERT INTO
  ___ (___)
VALUES
  (:___,  );
"""
]
query = insert_stations

path = 'TRL1_Database.sqlite'
# Execute function to insert values from Pandas database into SQLite.
sqlite_con_queries_inserts.execute_inserts(table, query, path)